In [5]:
import math
import random
import pygame
import tkinter as tk
from tkinter import messagebox

class options:
    
    # Theese are tweakable. Adjust to fit your needs.
    
    ScreenWidth = 1000
    ArenaSize = 50                      # In number of squares
    FPSLimit = 60
    SnakeSpeed = 4                     # I recommend integers from 1 to 10 and I dare you to score 10 points on 10.
    TurnsTillStarve = 300
    
    delay = 100 // SnakeSpeed
    SquareWidth = ScreenWidth // ArenaSize

class colors:
        
    gold = 255, 215, 0
    silver = 192, 192, 192
    black = 0, 0, 0
    green = 0, 128, 0
    red = 255, 0, 0
    cyan = 0, 255, 255
    white = 255, 255, 255
    
class square:
    
    def __init__(self, coords, color):
        
        self.coords = coords
        self.color = color
        self.vx = 0
        self.vy = -1

    def move(self, vx, vy):
        
        self.vx = vx
        self.vy = vy
        self.coords = (self.coords[0] + self.vx, self.coords[1] + self.vy)
 
    def draw(self, surface, eyes=False):
            
        SW = options.SquareWidth
        pygame.draw.rect(surface, self.color, (self.coords[0]*SW+1, self.coords[1]*SW+1, SW-1, SW-1))
        
        if eyes:
            centre = SW//2
            radius = 3
            circleMiddle = (self.coords[0]*SW+centre-radius, self.coords[1]*SW+8)
            circleMiddle2 = (self.coords[0]*SW+SW-radius*2, self.coords[1]*SW+8)
            pygame.draw.circle(surface, colors.black, circleMiddle, radius)
            pygame.draw.circle(surface, colors.black, circleMiddle2, radius)

class wall:
        
    def __init__(self):
        
        self.w = []
        self.R = options.ArenaSize
        
    def build(self):
        
        B = colors.black
    
        for j in range (self.R):
            self.w.append(square((j, 0), color=B))
            self.w.append(square((j, self.R - 1), color=B))
        
            if j != 0 and j != self.R - 1:
                self.w.append(square((0, j), color=B))
                self.w.append(square((self.R - 1, j), color=B))
                
            if 1.5 * self.R // 5 <= j < 3.5 * self.R // 5:
                self.w.append(square((self.R // 3, j), color=B))
                self.w.append(square((2 * self.R // 3, j), color=B))
                
    
    def draw(self, surface):
        
        for j in range (4*self.R+4*self.R//5-4):
            self.w[j].draw(surface)
    
class food:
    
    def __init__(self):
        
        self.R = options.ArenaSize
        self.numy = 0, 0
        self.yuck = 0, 0
        
    def RandomSnack(self, snake1, snake2, wall, food1, food2, food3, num=True):

        s1 = snake1.body
        s2 = snake2.body
    
        while True:
            x = random.randrange(1, self.R-1)
            y = random.randrange(1, self.R-1)
            
            condition1 = len(list(filter(lambda z:z.coords == (x, y), s1))) > 0
            condition2 = len(list(filter(lambda z:z.coords == (x, y), s2))) > 0
            condition3 = len(list(filter(lambda z:z.coords == (x, y), wall))) > 0
            condition4 = (x, y) == food1 or (x, y) == food2 or (x, y) == food3
            
            if condition1 or condition2 or condition3 or condition4:
                continue
                
            else:
                if num == True:
                    self.numy = x, y
                    self.nsquare = square((x, y), colors.red)
                    break

                elif num == False:
                    self.yuck = x, y
                    self.ysquare = square((x, y), colors.cyan)
                    break
    
    def draw(self, surface):
        
        self.nsquare.draw(surface=surface)
        self.ysquare.draw(surface=surface)
    
    
    
class snake:
    
    def __init__(self, coords, color):
        
        self.color = color
        self.start = coords
        self.body = []
        self.turns = {}
        self.head = square(self.start, self.color)
        self.body.append(self.head)
        self.vx = 0
        self.vy = 0
        self.RIP = False
        self.starve = options.TurnsTillStarve
 
    def move_arrows(self):
 
        self.keys = pygame.key.get_pressed()

        for key in self.keys:
            if self.keys[pygame.K_LEFT] and self.vx != 1 and self.vx != -1:
                self.vx = -1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_RIGHT] and self.vx != 1 and self.vx != -1:
                self.vx = 1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_UP] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = -1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_DOWN] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = 1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]
 
        for i, c in enumerate(self.body):
            p = c.coords[:]
            
            if p in self.turns:
                turn = self.turns[p]
                c.move(turn[0],turn[1])
                
                if i == len(self.body)-1:
                    self.turns.pop(p)
                
            else:
                c.move(c.vx, c.vy)
                
        self.starve -= 1
                
    def move_wasd(self):
        
        self.keys = pygame.key.get_pressed()
  
        for key in self.keys:
            if self.keys[pygame.K_a] and self.vx != 1 and self.vx != -1:
                self.vx = -1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_d] and self.vx != 1 and self.vx != -1:
                self.vx = 1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_w] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = -1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_s] and self.vy != 1 and self.vy != -1:
                self.vx = 0
                self.vy = 1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]
 
        for i, c in enumerate(self.body):
            p = c.coords[:]
            
            if p in self.turns:
                turn = self.turns[p]
                c.move(turn[0],turn[1])
                
                if i == len(self.body)-1:
                    self.turns.pop(p)
                
            else:
                c.move(c.vx, c.vy)
            
        self.starve -= 1
 
    def reset(self):
        
        self.head = square(self.start, self.color)
        self.body = []
        self.body.append(self.head)
        self.turns = {}
        self.vx = 0
        self.vy = 1
 
 
    def AddSquare(self):
        
        tail = self.body[-1]
        dx, dy = tail.vx, tail.vy
 
        if dx == 1 and dy == 0:
            self.body.append(square((tail.coords[0]-1,tail.coords[1]), self.color))
        elif dx == -1 and dy == 0:
            self.body.append(square((tail.coords[0]+1,tail.coords[1]), self.color))
        elif dx == 0 and dy == 1:
            self.body.append(square((tail.coords[0],tail.coords[1]-1), self.color))
        elif dx == 0 and dy == -1:
            self.body.append(square((tail.coords[0],tail.coords[1]+1), self.color))
 
        self.body[-1].vx = dx
        self.body[-1].vy = dy
        
    def RemoveSquare(self):
        if len(self.body) > 1:
            del self.body[-1]
        else:
            self.RIP = True

    def draw(self, surface):
        
        for i, c in enumerate(self.body):
            if i == 0:
                c.draw(surface, eyes=True)
            else:
                c.draw(surface)

class game:
    
    def __init__(self):
        
        self.quit = False
    
    def KeyboardInput(self):
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        if pygame.key.get_pressed()[pygame.K_ESCAPE]:
            self.quit = True
            
    def RedrawWindow(self, window, message1, message2):
        
        window.fill(colors.green)
        
        font = pygame.font.Font('freesansbold.ttf', 20)
        text1 = font.render(message1, True, colors.white, colors.green)
        text2 = font.render(message2, True, colors.white, colors.green)
        text3 = font.render('Speeed: ' + str(options.SnakeSpeed), True, colors.white, colors.green)
        TextRect1 = text1.get_rect()
        TextRect1.center = (options.ScreenWidth // 2, 3 * options.SquareWidth // 2)
        window.blit(text1, TextRect1)
        TextRect2 = text2.get_rect()
        TextRect2.center = (options.ScreenWidth // 2, 5 * options.SquareWidth // 2)
        window.blit(text2, TextRect2)
        TextRect3 = text3.get_rect()
        TextRect3.center = (options.ScreenWidth // 2, 7 * options.SquareWidth // 2)
        window.blit(text3, TextRect3)
        
        self.W.draw(window)
        self.s1.draw(window)
        self.s2.draw(window)
        self.F1.draw(window)
        self.F2.draw(window)
        
        pygame.display.update()

    
    def message_box(self, subject, content):
        
        root = tk.Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        messagebox.showinfo(subject, content)
        try:
            root.destroy()
        except:
            pass

    def reset(self):
        
        self.s1.reset()
        self.s2.reset()
        self.s1.starve = 300
        self.s2.starve = 300
        
    def eat(self, s):
        
        if s.body[0].coords == self.F1.numy:
            s.AddSquare()
            s.starve += 100
            self.F1.RandomSnack(self.s1, self.s2, self.W.w, self.F1.yuck, self.F2.numy, self.F2.yuck)
            
        elif s.body[0].coords == self.F1.yuck:
            s.RemoveSquare()
            s.starve += 100
            self.F1.RandomSnack(self.s1, self.s2, self.W.w, self.F1.numy, self.F2.numy, self.F2.yuck, num=False)
            
        elif s.body[0].coords == self.F2.numy:
            s.starve += 100
            s.AddSquare()
            self.F2.RandomSnack(self.s1, self.s2, self.W.w, self.F2.yuck, self.F1.numy, self.F1.yuck)
            
        elif s.body[0].coords == self.F2.yuck:
            s.starve += 100
            s.RemoveSquare()
            self.F2.RandomSnack(self.s1, self.s2, self.W.w, self.F2.numy, self.F1.numy, self.F1.yuck, num=False)
            
    def collision(self, s, x, render):

        d = 0
        
        if s.body[0].coords in list(map(lambda z:z.coords, self.s1.body[1:])):
            if render == True:
                if x == 1:
                    self.message = 'Ouroborus! Player ' + str(x) + ' bit his tail. \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\nPress OK to play again. You can always quit with ESCAPE.'
                elif x == 2:
                    self.message = 'Player ' + str(x) + ' is now a snack to Player 1. \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                self.note = 'Player ' + str(x) + ' lost!'
            d = 1

        elif s.body[0].coords in list(map(lambda z:z.coords, self.W.w)):
            if render == True:
                self.message = 'Tsk tsk tsk... Player ' + str(x) + ' should mind the suroundings more. \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                self.note = 'Player ' + str(x) + ' lost!'
            d = 2

        elif s.body[0].coords in list(map(lambda z:z.coords, self.s2.body[1:])):
            if render == True:
                if x == 1:
                    self.message = 'Player ' + str(x) + ' is now a snack to Player 2. \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                elif x == 2:
                    self.message = 'Ouroborus! Player ' + str(x) + ' bit his tail. \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\nPress OK to play again. You can always quit with ESCAPE.'
                self.note = 'Player ' + str(x) + ' lost!'
            d = 3

        elif s.RIP == True:
            if render == True:
                self.message = 'Player ' + str(x) + ' should\'ve taken the red pill. \n\nPlayer 1 score: ' + str(len(self.s1.body)-1) + \
                '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                self.note = 'Player ' + str(x) + ' lost!'
            d = 4
            s.RIP = False

        elif s.starve == 0:
            if render == True:
                self.message = 'Player ' + str(x) + ' starved to death. \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                '\nPlayer 2 score: ' + str(len(self.s2.body)-1) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                self.note = 'Player ' + str(x) + ' lost!'
            d = 5
        
        return d
    
    def CollisionHeadOn(self, render):
        
        if self.s1.body[0].coords == self.s2.body[0].coords:
            if render == True:
                self.message = 'Head-on! That had to hurt... \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                self.note ='R.I.P. poor snakes'
                self.message_box(self.note, self.message)
            self.reset()
 
    def play(self, two_players=False, render=True):
        
        self.R = options.ArenaSize
        screen_size = options.ScreenWidth, options.ScreenWidth
        self.W = wall()
        self.W.build()
        
        if two_players == True:
            self.s1 = snake((7*self.R//8, 7*self.R//8), colors.gold)
            self.s2 = snake((self.R//8, 7*self.R//8), colors.silver)
            
        else:
            self.s1 = snake((self.R//2, self.R//2), colors.gold)
            self.s2 = snake((-1, -1), colors.silver)
            
        self.F1 = food()
        self.F2 = food()
        self.F1.RandomSnack(self.s1, self.s2, self.W.w, self.F1.yuck, self.F2.numy, self.F2.yuck)
        self.F1.RandomSnack(self.s1, self.s2, self.W.w, self.F1.numy, self.F2.numy, self.F2.yuck, num=False)
        self.F2.RandomSnack(self.s1, self.s2, self.W.w, self.F2.yuck, self.F1.numy, self.F1.yuck)
        self.F2.RandomSnack(self.s1, self.s2, self.W.w, self.F2.numy, self.F1.numy, self.F1.yuck, num=False)
            
        if render == True:
            pygame.init() 
            window = pygame.display.set_mode(screen_size)
            pygame.display.set_caption('Eat or be eaten!')
            clock = pygame.time.Clock()
            #pygame.mixer.init()
            #pygame.mixer.music.load('C:/Users/Asas/Desktop/Python/Snak/GB.mp3') # Insert your favorite music here with path to it 
            #pygame.mixer.music.play(-1)
            
        while True: 
            if render == True:
                pygame.time.delay(options.delay)
                clock.tick(options.FPSLimit)
                self.KeyboardInput()
                    
            self.s1.move_arrows()
            self.eat(self.s1)
            death1 = self.collision(self.s1, 1, render)
            death2 = 0
            
            if self.s1.vx == -self.s2.vx or self.s1.vy == -self.s2.vy:
                self.CollisionHeadOn(render)
            
            if two_players == True:
                self.s2.move_wasd()
                self.eat(self.s2)
                death2 = self.collision(self.s2, 2, render)
                
            self.CollisionHeadOn(render)
                                               
            if death1 != 0 or death2 != 0:
                if death1 != 0 and death2 != 0:
                    if render == True:
                        self.message = 'Suspicious... both players suicided synchronically. \n\nPlayer 1 score: ' + str(len(self.s1.body)) + \
                        '\nPlayer 2 score: ' + str(len(self.s2.body)) + '\n\nPress OK to play again. You can always quit with ESCAPE.'
                        self.note = 'R.I.P. poor snakes'
                if render == True:
                    self.message_box(self.note, self.message)
                self.reset()
                                
            if self.quit == True:
                break 
                    
            if render == True:
                message1 = 'Player 1 score: ' + str(len(self.s1.body)) + '   Moves till starving: ' + str(self.s1.starve)
                message2 = 'Player 2 score: ' + str(len(self.s2.body)) + '   Moves till starving: ' + str(self.s2.starve) 
                self.RedrawWindow(window, message1, message2)
                
        if render == True:
            #pygame.mixer.music.stop
            pygame.quit()
            #sys.exit()


g = game()
g.play(two_players=False, render=True)